In [6]:
import pandas as pd
from datetime import datetime
df = d.read_csv("../DATA/Tokyo house price.csv",encoding="cp932")
#清空取引の事情等里面有内容的
filtered_df = df[df['取引の事情等'].isna()]p

# 筛选出'取引時期'列中包含2018年至2023年的数据
df['取引時期'] = filtered_df ['取引時期'].astype(str)
filtered_df = filtered_df [filtered_df ['取引時期'].str.contains(r'202[0-3]|201[6-9]', regex=True)]
filtered_df = filtered_df .query('種類 == "宅地(土地と建物)"')
filtered_df = filtered_df .query('今後の利用目的 == "住宅"')
# 使用这些做特征并清除所有空值
selected_columns = ['最寄駅：距離（分）', '取引価格（総額）', '面積（㎡）','建築年', '建物の構造','地区名',"建ぺい率（％）","容積率（％）",'取引時期']
selected_df = filtered_df[selected_columns]
selected_df = selected_df.dropna()

# 提取建筑年份的年份部分并转换为整数
selected_df['建築年'] = selected_df['建築年'].str.extract('(\d+)')
selected_df = selected_df.dropna()
selected_df['建築年'] = selected_df['建築年'].astype(int)
current_year = datetime.now().year
selected_df['建築年数'] = current_year - selected_df['建築年']
# selected_df = selected_df.drop("建築年",axis=1)

# 定义映射关系
distance_mapping = {
    '30分～60分': 30,
    '1H～1H30': 60,
    '1H30～2H': 90,
    '2H～': 120,
    '2,000㎡以上': 120
}

# 将文本值映射成数字
selected_df['最寄駅：距離（分）'] = selected_df['最寄駅：距離（分）'].map(lambda x: distance_mapping[x] if x in distance_mapping else int(x))

# 定义映射关系
distance_mapping = {
    '2,000㎡以上': 2000
}
# 将文本值映射成数字
selected_df['面積（㎡）'] = selected_df['面積（㎡）'].map(lambda x: distance_mapping[x] if x in distance_mapping else int(x))

filtered_data = selected_df[(selected_df['取引価格（総額）'] > 5000000) & (selected_df['取引価格（総額）'] <= 200000000)]
filtered_data = filtered_data[filtered_data['面積（㎡）'] < 200]
filtered_data = filtered_data[filtered_data['最寄駅：距離（分）'] < 30]

filtered_data['新耐震'] = (filtered_data['建築年数'] <= 43).astype(int)

In [4]:
filtered_data

,種類,価格情報区分,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時期,取引の事情等,建築年数,新耐震


In [ ]:
# 提取'市区町村名'和'地区名'
mapping = filtered_df[['市区町村名', '地区名']]

# 去重，以保证每个映射关系是唯一的
mapping = mapping.drop_duplicates()

# 可以将结果保存到一个新的CSV文件中
mapping.to_csv('city_district_mapping.csv', index=False)

print(mapping)

In [10]:

# 按行分割数据并处理每行
df = pd.read_csv('city_district_mapping.csv')
# 使用groupby和apply来创建字典
districts_by_city = df.groupby('市区町村名')['地区名'].apply(list).to_dict()

# 打印字典查看结果
print(districts_by_city)

{'あきる野市': ['秋川', '秋留', '網代', '油平', '雨間', '五日市', '伊奈', '入野', '牛沼', '小川', '小川東', '乙津', '上代継', '草花', '小中野', '小和田', '三内', '下代継', '菅生', '瀬戸岡', '高尾', '舘谷', '留原', '二宮', '二宮東', '野辺', '原小宮', '引田', '平沢', '平沢西', '深沢', '渕上', '山田', '戸倉', '横沢'], '三宅村': ['阿古', '伊ヶ谷', '伊豆', '神着', '坪田'], '三鷹市': ['井口', '井の頭', '大沢', '上連雀', '北野', '下連雀', '新川', '深大寺', '中原', '野崎', '牟礼'], '世田谷区': ['赤堤', '池尻', '宇奈根', '梅丘', '大蔵', '大原', '岡本', '奥沢', '尾山台', '粕谷', '鎌田', '上馬', '上北沢', '上祖師谷', '上野毛', '上用賀', '北烏山', '北沢', '喜多見', '砧', '給田', '経堂', '豪徳寺', '駒沢', '桜', '桜丘', '桜上水', '桜新町', '三軒茶屋', '下馬', '新町', '成城', '瀬田', '世田谷', '祖師谷', '代沢', '太子堂', '代田', '玉川', '玉川台', '玉川田園調布', '玉堤', '千歳台', '弦巻', '等々力', '中町', '野毛', '野沢', '八幡山', '羽根木', '東玉川', '深沢', '船橋', '松原', '三宿', '南烏山', '宮坂', '用賀', '若林'], '中央区': ['明石町', '入船', '新川', '新富', '築地', '月島', '佃', '日本橋蛎殻町', '日本橋茅場町', '日本橋小網町', '日本橋中洲', '日本橋人形町', '日本橋馬喰町', '日本橋箱崎町', '日本橋浜町', '日本橋横山町', '八丁堀', '東日本橋', '湊', '銀座', '日本橋久松町', '勝どき', '日本橋大伝馬町', '日本橋小舟町', '日本橋富沢町'], '中野区': ['新井', '江古田', '江原町', '上鷺宮', '上高田', '鷺宮'

In [11]:
# 加载地区数据
districts_data = pd.read_csv('web/city_district_mapping.csv')
# 创建市区町村名到地区名的映射字典
city_to_districts = districts_data.groupby('市区町村名')['地区名'].apply(list).to_dict()

In [ ]:
filtered_df.head()

In [ ]:
# 使用字典来组织数据
districts_by_city = {}
for item in mapping:
    city, district = item.split(',')  # 将字符串分割成两部分
    if city not in districts_by_city:
        districts_by_city[city] = []
    districts_by_city[city].append(district)

# 生成JavaScript对象
js_object = ',\n'.join(
    f"            '{city}': {districts}" for city, districts in districts_by_city.items()
)

# 输出JavaScript对象
print(f"var cityDistricts = {{\n{js_object}\n        }};")

In [ ]:
filtered_data.to_csv('./DATA/exported_data11.csv', index=False, encoding='utf-8')

In [ ]:
import joblib
#使用encoder编码建物の構造
from sklearn.preprocessing import LabelEncoder
# 确保列为字符串类型
for column in [ '建物の構造']:
    selected_df[column] = selected_df[column].astype(str)

# 创建LabelEncoder对象
label_encoder = LabelEncoder()

# 对每个object列进行标签编码
for column in [ '建物の構造']:
    selected_df[column] = label_encoder.fit_transform(selected_df[column])

print(selected_df)
joblib.dump(label_encoder, 'label_encoder.pkl')

In [ ]:
#使用encoder编码建物の構造
from sklearn.preprocessing import LabelEncoder
# 确保列为字符串类型
for column in ['地区名']:
    selected_df[column] = selected_df[column].astype(str)

# 创建LabelEncoder对象
label_encoder = LabelEncoder()

# 对每个object列进行标签编码
for column in ['地区名']:
    selected_df[column] = label_encoder.fit_transform(selected_df[column])

print(selected_df)

In [ ]:
df_onehot = selected_df.dropna(subset=['最寄駅：距離（分）'])

In [ ]:
print(selected_df['地区名'].isna().sum())

In [ ]:
#使用one-hot编码地区
from sklearn.preprocessing import OneHotEncoder
# 使用OneHotEncoder进行独热编码
# 使用 OneHotEncoder 进行独热编码
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_features = one_hot_encoder.fit_transform(selected_df[['地区名']])

# 获取独热编码后的特征名
feature_names = one_hot_encoder.get_feature_names_out(['地区名'])

# 将编码后的特征转换为 DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=feature_names, index=selected_df.index)

# 确认编码后的 DataFrame 中没有空值
print(encoded_df.isna().sum().sum())

# 合并原始数据和独热编码后的数据
df_onehot = pd.concat([selected_df, encoded_df], axis=1)

# 删除原始的 '地区名' 列
df_onehot = df_onehot.drop("地区名", axis=1)

joblib.dump(one_hot_encoder, 'one_encoder.pkl')

In [ ]:
print(feature_names)

In [ ]:
df_onehot['最寄駅：距離（分）'].unique()

In [ ]:
# 定义一个函数来去除离群值
# 四分位距（IQR）方法是一种常用的检测离群值的方法。它利用数据的第一四分位数（Q1）和第三四分位数（Q3）来定义一个范围，任何超出该范围的值都被视为离群值。
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

# 去除 '取引価格（総額）' 的离群值
data_cleaned = remove_outliers(df_onehot, '取引価格（総額）')

In [ ]:
import numpy as np
# 定义一个函数来去除离群值
# Z-Score 方法是通过计算每个数据点与均值的标准差来检测离群值的。通常，Z-Score 大于 3 或小于 -3 的数据点被视为离群值。
def remove_outliers_zscore(df, column):
    mean = df[column].mean()
    std = df[column].std()
    z_scores = (df[column] - mean) / std
    df_filtered = df[np.abs(z_scores) < 3]
    return df_filtered

# 去除 '取引価格（総額）' 的离群值
data_cleaned = remove_outliers_zscore(data_cleaned, '取引価格（総額）')

In [ ]:
filtered_data.info()

In [ ]:
filtered_data.columns